# Clustering Crypto

In [25]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [26]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv("crypto_data.csv", index_col = 0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [27]:
crypto_df['IsTrading'].value_counts()

True     1144
False     108
Name: IsTrading, dtype: int64

In [28]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [29]:
crypto_df['IsTrading'].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [30]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns = ['IsTrading'], inplace = True)

In [31]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace = True)

In [32]:
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [33]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [34]:
crypto_df['TotalCoinsMined'].describe()

count    5.320000e+02
mean     5.340456e+09
std      4.564568e+10
min      4.199995e+01
25%      8.359849e+06
50%      2.474397e+07
75%      1.867250e+08
max      9.899887e+11
Name: TotalCoinsMined, dtype: float64

In [35]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names = crypto_df.copy()
crypto_names.drop(columns = ['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply'], inplace=True)
crypto_names.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [36]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns = ['CoinName'], inplace=True)
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [37]:
#Isn't treating Total Coin Supply as a number
crypto_df['TotalCoinSupply'] = crypto_df['TotalCoinSupply'].apply(pd.to_numeric)

In [38]:
crypto_df['TotalCoinSupply'].describe()

count    5.320000e+02
mean     1.034523e+10
std      6.773697e+10
min      0.000000e+00
25%      2.100000e+07
50%      7.841760e+07
75%      5.000000e+08
max      1.000000e+12
Name: TotalCoinSupply, dtype: float64

In [39]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df)
X.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,8.400000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,2.200000e+07,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,2.100000e+08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X.values)
X_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [41]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.33114425,  1.08777262, -0.41281829],
       [-0.31448403,  1.08788353, -0.4130255 ],
       [ 2.29581638,  1.57719611, -0.4751978 ],
       ...,
       [ 0.3207143 , -2.34744386,  0.3119554 ],
       [-0.13949798, -2.06320012,  0.12139312],
       [-0.28753062,  0.82109427, -0.15846371]])

In [42]:
#Get index from X
index_x = list(X.index.values)

In [43]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data = X_pca, columns=['PC 1', 'PC 2', 'PC 3'])
pcs_df.index = index_x
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.331144,1.087773,-0.412818
404,-0.314484,1.087884,-0.413025
1337,2.295816,1.577196,-0.475198
BTC,-0.149740,-1.369351,0.117186
ETH,-0.156729,-2.052117,0.304150
LTC,-0.160013,-1.070827,0.009221
DASH,-0.398167,1.254529,-0.421660
XMR,-0.146235,-2.225052,0.304672
ETC,-0.155171,-2.052226,0.304141
ZEC,-0.139497,-2.063200,0.121393


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [44]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1,11))

for i in k:
    km = KMeans(n_clusters=i, random_state = 573)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\Christopher\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

The elbow is clearly at k = 4 when the bend goes mostly horizontal.

Running K-Means with `k=4`

In [45]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state = 573)

# Fit the model
model.fit(pcs_df)

# Predict clusters
model.labels_

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [47]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df, crypto_names], axis=1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.331144,1.087773,-0.412818,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.314484,1.087884,-0.413025,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.295816,1.577196,-0.475198,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.149740,-1.369351,0.117186,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.156729,-2.052117,0.304150,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.160013,-1.070827,0.009221,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.398167,1.254529,-0.421660,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.146235,-2.225052,0.304672,Monero,1
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.155171,-2.052226,0.304141,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.139497,-2.063200,0.121393,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [54]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=1000,
    hover_name="CoinName",
    hover_data=["Algorithm"]
)

fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [55]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply',
                                    'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [59]:
# Print the total number of tradable cryptocurrencies.
print("There are", len(clustered_df.index), "tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [61]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler()
scaled = scaler.fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [64]:
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.331144,1.087773,-0.412818,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.314484,1.087884,-0.413025,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.295816,1.577196,-0.475198,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.149740,-1.369351,0.117186,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.156729,-2.052117,0.304150,Ethereum,1


In [68]:
# Copy the clustered_df and drop columns not needed
plot_df = clustered_df.copy()
plot_df.drop(columns=['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply', 'PC 1', 'PC 2', 'PC 3'],
              inplace=True)

# Convert the array of scaled values into a dataframe with same index as clustered_df
temp_df = pd.DataFrame(scaled, columns = ['TotalCoinSupply', 'TotalCoinsMined'], index=list(scaled_df.index))

#Concat the two
plot_df = pd.concat([plot_df, temp_df], axis=1)

plot_df.head(10)

,CoinName,Class,TotalCoinSupply,TotalCoinsMined
42,42 Coin,0,4.200000e-11,0.000000
404,404Coin,0,5.320000e-04,0.001066
1337,EliteCoin,0,3.141593e-01,0.029576
BTC,Bitcoin,1,2.100000e-05,0.000018
ETH,Ethereum,1,0.000000e+00,0.000109
LTC,Litecoin,1,8.400000e-05,0.000064
DASH,Dash,0,2.200000e-05,0.000009
XMR,Monero,1,0.000000e+00,0.000017
ETC,Ethereum Classic,1,2.100000e-04,0.000115
ZEC,ZCash,1,2.100000e-05,0.000007


In [70]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', by='Class', hover_cols = ['CoinName'])

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)